<a href="https://colab.research.google.com/github/ALLULOHITHA/FMML-LABS/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits typically provides more consistent and reliable results. This is a key principle behind techniques like k-fold cross-validation. Here's why it works and its benefits:


---

1. Why Averaging Improves Consistency

Mitigates Variance from Data Splits:

Single train-test splits can lead to variability because the performance depends heavily on how the data was divided.

By averaging results over multiple splits (e.g., k folds), you smooth out the randomness from individual splits.


Utilizes the Entire Dataset:

In k-fold cross-validation, every data point gets to be in the training and validation sets exactly once. This ensures no data is wasted and reduces bias.




---

2. How It Works

Split the dataset into k folds (e.g., 5 or 10).

Train and validate the model k times, each time using a different fold for validation and the rest for training.

Compute the average validation accuracy over all k folds to get a robust estimate of the model’s performance.



---

3. Benefits

Reduced Risk of Overfitting/Underfitting:

Validation accuracy from multiple splits is less likely to be skewed by a particularly easy or hard split.


Better Generalization:

The averaged result is more reflective of the model's performance on unseen data.


More Stable Hyperparameter Tuning:

When testing different configurations, cross-validation gives more reliable feedback, preventing overfitting to a single validation set.




---

4. When Might It Not Help?

Large Datasets:

If the dataset is very large and representative, a single split may already give stable results, and cross-validation might not add much benefit.


Imbalanced Datasets:

If classes are imbalanced, some folds may have disproportionate class distributions. Use stratified k-fold cross-validation to ensure class balance in each split.




---

Conclusion

Averaging validation accuracy across multiple splits (e.g., through cross-validation) is a robust strategy that improves result consistency, especially for small or medium-sized datasets or when data splits are noisy. Would you like guidance on implementing this approach?


2. Does it give more accurate estimate of test accuracy?
Averaging validation accuracy across multiple splits (e.g., through cross-validation) does provide a more accurate estimate of test accuracy, but with some caveats. Here's a breakdown:


---

1. Why Cross-Validation Improves Accuracy Estimates

Reduces Variance from Single Splits:

A single train-test split can result in either overestimating or underestimating test accuracy due to random variations in the split.

Cross-validation averages the performance across multiple splits, reducing this variance and giving a more representative estimate of how the model will perform on unseen data.


Better Use of Data:

Every data point is used for both training and validation in cross-validation. This ensures that the estimate reflects the performance across the entire dataset, not just one portion.


Mitigates Bias:

Some splits may create biased training or validation sets (e.g., one split might have many "easy" examples). Cross-validation balances this out.




---

2. Is It Closer to Test Accuracy?

Yes, in most cases, cross-validation gives a better estimate of the test accuracy compared to a single train-validation split because:

It simulates how well the model generalizes by testing on multiple validation sets.

It accounts for variability in the training data.


However:

Test accuracy may still differ slightly because the actual test set is unseen during cross-validation and may have slightly different properties than the training data.

If the test set distribution differs significantly (e.g., different noise, data imbalance), cross-validation won't perfectly predict test accuracy.




---

3. Situations Where Cross-Validation May Not Align with Test Accuracy

Data Leakage:

If features or labels in the training data overlap improperly (e.g., time series data not split correctly), the validation accuracy will overestimate test performance.


Imbalanced or Small Datasets:

For very small datasets or highly imbalanced classes, even cross-validation can struggle to approximate true test accuracy.


Hyperparameter Tuning Bias:

If cross-validation is used extensively for hyperparameter tuning, it may overfit to the validation folds, leading to slightly optimistic estimates.




---

4. How to Get the Most Accurate Estimate

Nested Cross-Validation:

Use an outer loop for estimating test accuracy and an inner loop for hyperparameter tuning. This ensures the validation set is not indirectly influencing hyperparameter selection.


Stratified Cross-Validation:

Ensures that class proportions in the training and validation sets match those in the dataset, leading to more representative estimates.


Hold-Out Test Set:

After cross-validation, evaluate on a completely independent test set for the final accuracy estimate. This gives the best measure of real-world performance.




---

Conclusion

Cross-validation provides a more accurate and stable estimate of test accuracy compared to a single train-validation split, but the actual test accuracy may still differ slightly depending on the test set characteristics. For critical applications, always use a final independent test set to confirm performance.

Would you like an example or implementation for cross-validation?


3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The number of iterations (or folds in cross-validation) does affect the quality of the estimate, but whether a higher number of iterations gives a better estimate depends on several factors. Here's an explanation:


---

1. Effect of Increasing the Number of Iterations

Better Variance Reduction:

A higher number of iterations (e.g., more folds in k-fold cross-validation) generally reduces variance in the estimate because the model is tested on more diverse subsets of the data.

Example: 10-fold cross-validation typically produces more stable and reliable results than 3-fold cross-validation.


More Data in Training Sets:

With more iterations, each training set uses a larger proportion of the data. This helps models learn better representations and may improve the reliability of the accuracy estimate.


Diminishing Returns:

After a certain point (e.g., very high k in k-fold), the improvement in the accuracy estimate becomes negligible. For instance, moving from 10-fold to 20-fold cross-validation provides only a minor improvement.




---

2. Trade-offs with Higher Iterations

Computational Cost:

More iterations mean more training and validation cycles. This increases computational cost, especially for large datasets or complex models.


Potential for Overfitting to Validation Folds:

With very high iterations (e.g., leave-one-out cross-validation where k = number of data points), the model might overfit to small validation sets, especially if the dataset has noise or outliers.


Instability in Small Datasets:

In small datasets, higher iterations may lead to unstable estimates as small validation sets (e.g., single data points in leave-one-out) can be unrepresentative.




---

3. When Do Higher Iterations Give Better Estimates?

Large, Homogeneous Datasets:

With enough data, higher iterations (e.g., 10- or 20-fold cross-validation) can yield better estimates because subsets are more representative of the whole dataset.


When Data is Limited:

Leave-one-out cross-validation (LOOCV) maximizes training data usage and can provide better estimates for very small datasets, albeit at higher computational cost.


Hyperparameter Sensitivity:



4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Yes, increasing the number of iterations (e.g., in k-fold cross-validation) can help deal with a very small training or validation dataset to some extent, but there are limitations. Here's how it works and the potential issues:


---

1. How Increasing Iterations Helps with Small Datasets

Maximizes Data Usage:

In k-fold cross-validation, as the number of folds (k) increases, the size of the training set per fold grows because fewer samples are held out for validation.

In extreme cases, leave-one-out cross-validation (LOOCV) (k = number of samples) uses almost the entire dataset for training in each iteration, maximizing the training data while testing on one sample at a time.


More Robust Estimates:

With small datasets, a single train-validation split can lead to highly variable results due to the small size of the validation set. By averaging results across multiple folds, the estimate becomes more stable.




---

2. Trade-Offs of High Iterations with Small Datasets

While increasing iterations can help, there are limitations:

Overfitting to Validation Folds:

With small datasets, the model may overfit to the training data during each iteration, especially for complex models, leading to overly optimistic results.


Sensitivity to Noise and Outliers:

If the dataset is very small, each validation fold may have significant noise or outliers, making the results less reliable. This is particularly problematic with LOOCV, where each fold has only one validation sample.


Computational Cost:

Increasing iterations adds computational burden, especially with LOOCV, where the model is trained as many times as there are data points.




---

3. Alternatives to Handle Small Datasets

Data Augmentation:

For small datasets, consider generating synthetic data (e.g., using data augmentation for images or oversampling techniques like SMOTE for tabular data) to artificially increase dataset size.


Stratified Cross-Validation:

Use stratified k-fold cross-validation to ensure class proportions are preserved in each fold, which is critical for imbalanced small datasets.


Bootstrap Sampling:

Instead of cross-validation, bootstrap sampling repeatedly resamples the dataset with replacement, creating multiple training sets. This allows validation on unused samples.


Simpler Models:

Use less complex models (e.g., linear regression, logistic regression) that are less prone to overfitting, as these work better with small datasets.


Regularization:

Apply regularization techniques (e.g., L1/L2 regularization) to prevent overfitting when the training dataset is small.




---

4. When Increasing Iterations Works Best

Homogeneous Data:

If the dataset is small but representative of the target population (low noise or outliers), increasing iterations (e.g., 10-fold or LOOCV) can provide robust estimates.


Simple Models:

If the model is simple and unlikely to overfit, high iterations can help utilize all available data for training.


Few Outliers:

With minimal noise or outliers, higher iterations provide better estimates without skewing results.




---

Conclusion

Increasing iterations can mitigate the limitations of small datasets by maximizing data usage and providing more stable estimates. However, it is not a perfect solution, as it may amplify issues like overfitting or sensitivity to noise. Combining high iterations with strategies like data augmentation, simpler models, and regularization is the best approach.

Would you like me to demonstrate or simulate one of these strategies?

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.